In [1]:
import pandas as pd

In [2]:
import pysam
import os

In [3]:
refSeq = pd.read_table('./refseq.genes.ncbi37')
refSeq = refSeq[['chrom','name', 'strand', 'exonCount', 'exonStarts', 'exonEnds', 'exonFrames']]

In [4]:
refSeq.head(10)

,chrom,name,strand,exonCount,exonStarts,exonEnds,exonFrames
0,chr1,NM_032291,+,25,"66999824,67091529,67098752,67101626,67105459,6...","67000051,67091593,67098777,67101698,67105516,6...","0,1,2,0,0,0,1,0,0,0,1,2,1,1,1,1,0,1,1,2,2,0,2,..."
1,chr1,NM_052998,+,12,"33546713,33546988,33547201,33547778,33549554,3...","33546895,33547109,33547413,33547955,33549728,3...","-1,-1,-1,0,0,0,2,2,0,1,0,2,"
2,chr1,NM_001145278,+,8,"16767166,16770126,16774364,16774554,16775587,1...","16767270,16770227,16774469,16774636,16775696,1...","0,2,1,1,2,0,1,2,"
3,chr1,NM_001145277,+,7,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...","0,2,1,1,2,0,1,"
4,chr1,NM_001080397,+,8,"8384389,8385357,8385877,8390268,8395496,839787...","8384786,8385450,8386102,8390996,8395650,839805...","0,1,1,1,0,1,1,0,"
5,chr1,NM_018090,+,8,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...","0,2,1,1,2,0,1,2,"
6,chr1,NM_013943,+,6,"25071759,25124232,25140584,25153500,25166350,2...","25072116,25124342,25140710,25153607,25166532,2...","0,0,2,2,1,0,"
7,chr1,NM_032785,-,14,"48998526,49000561,49005313,49052675,49056504,4...","48999965,49000588,49005410,49052838,49056657,4...","2,2,1,0,0,2,1,1,0,2,0,1,1,0,"
8,chr1,NM_001195684,-,18,"92145899,92161228,92163645,92174219,92177799,9...","92149414,92161336,92163687,92174340,92178099,9...","1,1,1,0,0,0,0,0,1,0,2,1,0,0,1,0,-1,-1,"
9,chr1,NM_001195683,-,17,"92145899,92161228,92163645,92174219,92177799,9...","92149414,92161336,92163687,92174340,92178099,9...","1,1,1,0,0,0,0,0,1,0,2,1,0,0,1,0,-1,"


In [5]:
codonMap = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L",
    "TCT":"S", "TCC":"s", "TCA":"S", "TCG":"S",
    "TAT":"Y", "TAC":"Y", "TAA":"STOP", "TAG":"STOP",
    "TGT":"C", "TGC":"C", "TGA":"STOP", "TGG":"W",
    "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
    "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
    "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
    "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R",
    "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M",
    "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
    "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K",
    "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R",
    "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
    "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
    "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
    "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

In [6]:
def ReverseComplement(Text):
    reversedComplement = ""
    for x in range(0,len(Text)):
        if Text[x] == "A":
            curr = "T"
        elif Text[x] == "T":
            curr = "A"
        elif Text[x] == "G":
            curr = "C"
        else:
            curr = "G"
        reversedComplement = curr + reversedComplement
    return reversedComplement

In [23]:
def ExtractExonSeqs(data):
    exonSeqs = [""]*len(data)
    extractSeq = pysam.FastaFile('./chromFa/' + 'chr1' +'.fa')
    
    for i in range(len(data)):
        start = data['exonStarts'][i][:-1].split(',')
        end = data['exonEnds'][i][:-1].split(',')
        
        if i != 0 and data['chrom'][i] != data['chrom'][i-1]:
            extractSeq = pysam.FastaFile('./chromFa/' + data['chrom'][i] +'.fa')
        s = ""
        readFrame = data['exonFrames'][i][:-1].split(',')
        
        for j in range(len(start)):
            if readFrame[j] == str(-1):
                continue
            s += extractSeq.fetch("", int(start[j]), int(end[j]), data['chrom'][i])
        
        exonSeqs[i] = s
        
        if data['strand'][i] == '-':
            exonSeqs[i] = ReverseComplement(s)
    
    return exonSeqs

In [8]:
def Calculatelength(data):
    length = [0]*len(data)
    for i in range(len(data)):
        start = data['exonStarts'][i][:-1].split(',')
        end = data['exonEnds'][i][:-1].split(',')
        length[i] = [0]*len(start)
        for j in range(len(start)):
            length[i][j] = int(end[j]) - int(start[j])
    return length

In [9]:
exonLength = Calculatelength(refSeq)

In [24]:
exonSeqs = ExtractExonSeqs(refSeq)

In [25]:
exonSeqs

['...',
 '',
 '',
 '',
 '',
 'CCGGCATCCGGGATCCCTACGTCCCGCGTCCCCCGAGCGCTCGGAGCCTACGCGCCCAGCGCTACCGAAACCCAGAGTCCTGCGCCCTGGAGTCCCCGCGCCCCGGAGCCCGAGCACCCGGGAGTCCCGAGCCTCGCGCCCCGGAGTGCCCGAGCCTGCGCCGCCGCACCCGGATACCCCGCGTCCCCGCGAGCTGCCGAGGGGGGGGGGGGGGGGGGGGGGGACAGTACCGCCTTCCTCCCCTCTGTCCGCGCCATGGCCGCCCCCGACCTGTCCACCAACCTCCAGGAGGAGGCCACCTGCGCCATCTGCCTCGACTACTTCACGGATCCGGTGATGACCGACTGCGGCCACAACTTCTGCCGCGAGTGCATCCGGCGCTGCTGGGGCCAGCCCGAGGGCCCGTACGCGTGCCCCGAGTGCCGCGAGCTGTCCCCGCAGAGGAACCTGCGGCCCAACCGCCCGCTTGCTAAGATGGCCGAGATGGCGCGGCGCCTGCACCCGCCGTCGCCGGTCCCGCAGGGCGTGTGCCCCGCGCACCGCGAGCCACTGGCCGCCTTCTGTGGCGACGAGCTGCGCCTCCTGTGTGCGGCCTGCGAGCGCTCTGGGGAGCACTGGGCGCACCGCGTGCGGCCGCTGCAGGACGCGGCCGAAGACCTCAAGGCGAAGCTGGAGAAGTCACTGGAGCATCTCCGGAAGCAGATGCAGGATGCGTTGCTGTTCCAAGCCCAGGCGGATGAGACCTGCGTCTTGTGGCAGAAGATGGTGGAGAGCCAGCGGCAGAACGTGCTGGGTGAGTTCGAGCGTCTTCGCCGTTTGCTGGCAGAGGAGGAGCAGCAGCTGCTGCAGAGGCTGGAGGAGGAGGAGCTGGAGGTGCTGCCCCGGCTGCGGGAGGGCGCAGCCCACCTAGGCCAGCAGAGCGCCCACCTAGCTGAGCTCATCGCCGAGCTCGAGGGCCGCTGCCAGCTGC

In [26]:
def Translation(codonmap, seqs):
    threeReadingFrame = []
    
    # 3 reading frame
    for x in range(3):
        aa = []
        for i in range(x, len(seqs)):
            aminoAcid = ""
            for j in range(0, len(seqs[i])-2,3):
                
                # edge cases needed to be checked
                if seqs[i][j:j+3].upper() not in codonmap:
                    break
                    
                codon = codonmap[seqs[i][j:j+3].upper()]
                if codon == "STOP":
                    break
                aminoAcid += codon
            aa.append(aminoAcid)
    threeReadingFrame.append(aa)
    return threeReadingFrame

In [27]:
transcript = Translation(codonMap, exonSeqs)    

In [28]:
transcript

[['ETWSESRTAsVRRTEERWKSPEVRWAPGVAMEESGAAEWQLDQPSWSGRLRITAKGQMAYIKLEDRTSGELFAQAPVDQFPGTAVESVTDsSRYFVIRIEDGNGRRAFIGIGFGDRGDAFDFNVALQDHFKWVKQQCEFAKQAQNPDQGPKLDLGFKEGQTIKLNIANMKKKEGAAGNPRVRPASTGGLSLLPPPPGGKTsTLIPPPGEQLAVGGsLVQPAVAPSSGGAPVPWPQPNPATADIWGDFTKSTGSTsSQTQPGTGWVQF',
  'ETWSESRTAsVRRTEERWKSPEVRWAPGVAMEESGYESVLCVKPDVHVYRIPPRATNRGYRAAEWQLDQPSWSGRLRITAKGQMAYIKLEDRTSGELFAQAPVDQFPGTAVESVTDsSRYFVIRIEDGNGRRAFIGIGFGDRGDAFDFNVALQDHFKWVKQQCEFAKQAQNPDQGPKLDLGFKEGQTIKLNIANMKKKEGAAGNPRVRPASTGGLSLLPPPPGGKTsTLIPPPGEQLAVGGsLVQPAVAPSSDQLPARPSQAQAGsSSDLSTVFPHVTSGKALPHLGQRKEDEALLSWPVFGA',
  'MIPAASSTPPGDALFPSVAPQDFWRsQVTGYSGsVTRHLSHRANNFKRHPKRRKCIRPsPPPPPNTPCPLELVDFGDLHPQRsFRELLFNGCILFGIEFSYAMETAYVTPVLLQMGLPDQLYSLVWFISPILGFLLQPLLGAWSDRCTSRFGRRRPFILVLAIGALLGLSLLLNGRDIGIALADVTGNHKWGLLLTVCGVVLMDFSADSADNPSHAYMMDVCSPADQDRGLNIHALLAGLGGGFGYVVGGIHWDKTGFGRALGGQLRVIYLFTAVTLSVTTVLTLVSIPERPLRPPSEKRAAMKSPSLPLPPsPPVLPEEGPGDSLPSHTATNFsSPISPPSPLTPKYGSFISRDSsLTGISEFASsFGTANIDSVLIDCFTGGHDSYLAIPGSVPRPPISVSFPRAPDGFYRQD

In [ ]:
def Rescue(data, seqs):
    
    possibleRescue = [0]*len(data)
    exonLen = Calculatelength(data)
    for i in range(len(data)):
        readFrame = data['exonFrames'][i][:-1].split(',')
        exonNum = data['exonCount'][i]
        possibleRescue[i] = [0]*exonNum
        
        # delete the next exon to rescue
        for j in range(exonNum-1):  
            if readFrame[j] != str(-1):
                possibleRescue[i][j] = [0]*exonLen[i][j]
                exon = seq[i][j]
                
                for k in range(exonLen[i][j]):
                    possibleRescue[i][j][k] = [0]*2
                    # left to check new possible stop code
                    
                    
                    # frameshit +1
                    if exonLen[i][j+1] == 1:
                        possibleRescue[i][j][k][1] = 1
                    # frameshit +2
                    elif exonLen[i][j+1] == 2:
                        possibleRescue[i][j][k][1] = 2
                        
        # delete the previous exon to rescue                
        for j1 in range(1, exonNum): 
            if readFrame[j1] != str(-1):
                exon1 = seq[i][j1] 
                for k1 in range(exonLen[i][j1]):
                    # frameshit +1
                    if exonLen[i][j-1] == 1:
                        possibleRescue[i][j1][k1][2] = 1
                    # frameshit +2
                    elif exonLen[i][j-1] == 2:
                        possibleRescue[i][j1][k1][2] = 2
                        
    return possibleRescue